In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
import statsmodels.api as sm
from stargazer.stargazer import Stargazer, LineLocation
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from joblib import dump, load
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,ConfusionMatrixDisplay,precision_score,recall_score,PrecisionRecallDisplay
from scipy.stats import ttest_ind
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [24]:
df = pd.read_csv('InsuranceRiskRegression23.csv')

print(df.head())
print(df.info())

   Female  Own_car  Own_property  Work_phone  Phone  Email  Unemployed  \
0       1        1             0           0      1      1           1   
1       0        0             1           0      0      0           0   
2       0        0             0           1      1      0           0   
3       0        1             0           0      0      0           0   
4       0        0             1           0      0      0           1   

   Num_children  Num_family  Account_length  Total_income        Age  \
0             0           2              40      115290.0  63.483850   
1             0           2              57      360000.0  47.286392   
2             0           2              21      270000.0  42.872886   
3             0           2              31      270000.0  40.452576   
4             0           2              21      292500.0  59.089509   

   Years_employed           Income_type                 Education_type  \
0        0.000000             Pensioner  Seconda

In [25]:
print(df2.isna().sum())

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [26]:
# Created a hold out test set that you will use to evaluate the performance of models.
# Created a binary variable called 'Payout' that equals 1 if the client required a payout and 0 if not.

# Payout = 1 if payout > 0
df['Payout'] = (df['Insurance Payout'] > 0).astype(int)

# Select a few initial features
features = ['Age', 'Total_income', 'Years_employed', 'Num_children', 'Own_car', 'Unemployed']
X = df[features]
y = df['Payout']

# Train/test split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
## Create the best classification model 
## that you can that generate predictions of whether the client needed a payout.

# Fit logistic regression
log_model = LogisticRegression(max_iter=1000, random_state=42)
log_model.fit(X_train, y_train)

# Predict
log_preds = log_model.predict(X_test)

In [28]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Hyperparameter Grid
rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10]
}

# Grid Search for Random Forest
rf_grid = GridSearchCV(RandomForestClassifier(random_state=42),
                       rf_params, cv=3, scoring='recall', n_jobs=-1)
rf_grid.fit(X_train, y_train)

# Best model
best_rf = rf_grid.best_estimator_
rf_preds = best_rf.predict(X_test)


In [30]:
from sklearn.ensemble import GradientBoostingClassifier

# Gradient Boosting Hyperparameter Grid
gb_params = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

# Grid Search for Gradient Boosting
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state=42),
                       gb_params, cv=3, scoring='recall', n_jobs=-1)
gb_grid.fit(X_train, y_train)

# Best model
best_gb = gb_grid.best_estimator_
gb_preds = best_gb.predict(X_test)

In [31]:
from sklearn.metrics import classification_report

print("Logistic Regression Classification Report:")
print(classification_report(y_test, log_preds))

print("\nRandom Forest Best Params:", rf_grid.best_params_)
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_preds))

print("\nGradient Boosting Best Params:", gb_grid.best_params_)
print("Gradient Boosting Classification Report:")
print(classification_report(y_test, gb_preds))


🔹 Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       518
           1       0.97      0.98      0.97       682

    accuracy                           0.97      1200
   macro avg       0.97      0.97      0.97      1200
weighted avg       0.97      0.97      0.97      1200


🔹 Random Forest Best Params: {'max_depth': 10, 'n_estimators': 100}
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97       518
           1       0.97      0.98      0.97       682

    accuracy                           0.97      1200
   macro avg       0.97      0.97      0.97      1200
weighted avg       0.97      0.97      0.97      1200


🔹 Gradient Boosting Best Params: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

     

I used Logistic Regression, Random Forest Classifier, and Gradient Boosting Classifier.

All three models, as seen in the code above, score an accuracy of 97%. On top of that they also score the same in recall:
Class 0 recall = 96%, of all clients who truly didn’t need a payout, 96% were correctly predicted as no payout.
Class 1 recall = 98%, of all clients who truly needed a payout, 98% were correctly predicted as needing one.

I believe these three models are the best because I used logistic regression as a baseline model to see what it would score. Random Forest Classifier and Gradient Boosting Classifier are usually better because they have more hyperparameters, which I tuned with GridSearch. This ensures that I have the best combination of hyperparameters for these two models. I didn't create a very big grid to save time, but still the accuracy is very high.
I focused on recall because for an insurance company this would be a very important metric. Recall measures if the model can avoid false negatives. This is helpful information for an insurance company because a false negative is a client who is predicted to not be needing a payout, but ends up needing a payout. Which is not what the insurance company wants. Missing a client who will require a payout is more costly than falsely predicting someone who won’t.

I will use Gradient Boosting model for insurance payout calculation

In [33]:
df_paid = df[df['Insurance Payout'] > 0].copy()

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df_paid, drop_first=True)

# Define x and y
X_reg = df_encoded.drop(columns=['Insurance Payout', 'Payout'])
y_reg = df_encoded['Insurance Payout']

# split
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)


In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error

# polynomials degree 1 through 3
best_rmse = float('inf')
best_degree = None
best_lin_model = None
lin_preds = None

for deg in range(1, 4):
    model = make_pipeline(PolynomialFeatures(degree=deg, include_bias=False), LinearRegression())
    model.fit(Xr_train, yr_train)
    preds = model.predict(Xr_test)
    rmse = mean_squared_error(yr_test, preds, squared=False)
    
    if rmse < best_rmse:
        best_rmse = rmse
        best_degree = deg
        best_lin_model = model
        lin_preds = preds

print(f"Best Linear Regression Degree: {best_degree}")



Best Linear Regression Degree: 2


In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Hyperparameter grid
rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10]
}

# Grid Search
rf_grid = GridSearchCV(RandomForestRegressor(random_state=42),
                       rf_params, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
rf_grid.fit(Xr_train, yr_train)

# Best model
best_rf_reg = rf_grid.best_estimator_
rf_preds = best_rf_reg.predict(Xr_test)

print("Random Forest Best Params:", rf_grid.best_params_)


Random Forest Best Params: {'max_depth': 10, 'n_estimators': 100}


In [37]:
from sklearn.ensemble import GradientBoostingRegressor

# Hyperparameter grid
gb_params = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

# Grid Search
gb_grid = GridSearchCV(GradientBoostingRegressor(random_state=42),
                       gb_params, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
gb_grid.fit(Xr_train, yr_train)

# Best model
best_gb_reg = gb_grid.best_estimator_
gb_preds = best_gb_reg.predict(Xr_test)

print("Gradient Boosting Best Params:", gb_grid.best_params_)


Gradient Boosting Best Params: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


In [38]:
def evaluate_reg(name, y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{name} → RMSE: {rmse:.2f}, MAE: {mae:.2f}")

evaluate_reg(f"Polynomial Linear Regression (Degree {best_degree})", yr_test, lin_preds)
evaluate_reg("Random Forest Regressor", yr_test, rf_preds)
evaluate_reg("Gradient Boosting Regressor", yr_test, gb_preds)

Polynomial Linear Regression (Degree 2) → RMSE: 203.82, MAE: 145.38
Random Forest Regressor → RMSE: 245.89, MAE: 187.54
Gradient Boosting Regressor → RMSE: 196.30, MAE: 146.93


I ran the same three models but then as a regression model.

The polynomial regression model with degree = 2 performed the best out of the different degrees between 1 and 3.
The RMSE is 203.82.
The Random forest regression model scored an RMSE of 245.89 and this was tuned using GridSearch again to ensure that the best possible combinations of parameters would be found.
The Gradient Boosting Regressor performed best with an RMSE of 196.30, slightly outpeforming the polynomial regression model. This model was also tuned using GridSearch.

In the code below I will use both classification model and regression model to calculate the insurance payout. 
The classification model is used on a new client dataset and it will predict who will need a payout.
The regression model is used to predict the size of the payout.
 
To calculate the insurance premium for this pool of clients I used the following calculation:

PREMIUM = AVERAGE PAYOUT AMOUNT * PROBABILITY OF A PAYOUT * LOADING FACTOR. Where a loading factor of 1.2 was used.

In [54]:
df_new = pd.read_csv('InsuranceRiskRegression_NewClients.csv')

print(df_new.head())
print(df_new.info())

   Female  Own_car  Own_property  Work_phone  Phone  Email  Unemployed  \
0       0        1             1           0      0      0           0   
1       1        1             0           1      1      0           0   
2       0        1             1           0      0      0           1   
3       0        0             1           0      1      0           1   
4       1        1             1           0      1      0           0   

   Num_children  Num_family  Account_length  Total_income        Age  \
0             2           4               8       90000.0  31.425697   
1             0           2              52      144000.0  50.489743   
2             0           2               3      180000.0  57.257849   
3             0           2              19       85500.0  64.253202   
4             0           2              39      135000.0  61.715162   

   Years_employed Income_type                 Education_type Family_status  \
0        4.514809     Working  Secondary / s

In [55]:
# One-hot encode training data
df_paid = df[df['Insurance Payout'] > 0].copy()
df_train_encoded = pd.get_dummies(df_paid, drop_first=True)
X_train_encoded = df_train_encoded.drop(columns=['Insurance Payout', 'Payout'], errors='ignore')

# Save training feature columns
train_columns = set(X_train_encoded.columns)

# One-hot encode new client data
df_new_encoded = pd.get_dummies(df_new.copy(), drop_first=True)
new_columns = set(df_new_encoded.columns)

# Add missing columns to new client data
missing_cols = list(train_columns - new_columns)
for col in missing_cols:
    df_new_encoded[col] = 0

# Reorder to match training columns
df_new_encoded = df_new_encoded[sorted(train_columns)]

In [59]:
# Classification
df_class = pd.get_dummies(df.copy(), drop_first=True)

y_class = df_class['Payout']
X_class = df_class.drop(columns=['Insurance Payout', 'Payout'])

train_columns_classifier = X_class.columns

# Split
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)

# Retrain classifier
best_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
best_gb.fit(Xc_train, yc_train)



GradientBoostingClassifier(random_state=42)

In [60]:
# Regression and use only payout > 0
df_paid = df[df['Insurance Payout'] > 0].copy()
df_reg = pd.get_dummies(df_paid, drop_first=True)

y_reg = df_reg['Insurance Payout']
X_reg = df_reg.drop(columns=['Insurance Payout', 'Payout'])

train_columns_regressor = X_reg.columns

# Split
Xr_train, Xr_test, yr_train, yr_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Retrain regressor
from sklearn.ensemble import GradientBoostingRegressor
best_gb_reg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
best_gb_reg.fit(Xr_train, yr_train)



GradientBoostingRegressor(max_depth=5, random_state=42)

In [62]:
# For classification model
df_new_encoded_clf = df_new_encoded[train_columns_classifier]
df_new_orig['Predicted_Payout_Needed'] = best_gb.predict(df_new_encoded_clf)

# For regression model
df_new_encoded_reg = df_new_encoded[train_columns_regressor]
df_new_orig['Predicted_Payout_Amount'] = best_gb_reg.predict(df_new_encoded_reg)


In [65]:
## Probability of payout in the sample
payout_prob = df_new_orig['Predicted_Payout_Needed'].mean()
print(f"Probability of payout in this sample: {payout_prob:.4f}")

Probability of payout in this sample: 0.6132


In [66]:
avg_payout = df_new_orig.loc[df_new_orig['Predicted_Payout_Needed'] == 1, 'Predicted_Payout_Amount'].mean()

# Loading factor
loading_factor = 1.2

# Final premium calculation
premium = avg_payout * payout_prob * loading_factor
print(f"Recommended Insurance Premium: ${premium:.2f}")


Recommended Insurance Premium: $2361.53
